# BentoML Example: Fast AI with Tabular data

**BentoML makes moving trained ML models to production easy:**

* Package models trained with **any ML framework** and reproduce them for model serving in production
* **Deploy anywhere** for online API serving or offline batch serving
* High-Performance API model server with *adaptive micro-batching* support
* Central hub for managing models and deployment process via Web UI and APIs
* Modular and flexible design making it *adaptable to your infrastrcuture*

BentoML is a framework for serving, managing, and deploying machine learning models. It is aiming to bridge the gap between Data Science and DevOps, and enable teams to deliver prediction services in a fast, repeatable, and scalable way.
Before reading this example project, be sure to check out the [Getting started guide](https://github.com/bentoml/BentoML/blob/master/guides/quick-start/bentoml-quick-start-guide.ipynb) to learn about the basic concepts in BentoML.


This notebook is based on fastai v1's cours v3 lesson 4.  It will train a model that predict salary range base on the data we provided.


![Impression](https://www.google-analytics.com/collect?v=1&tid=UA-112879361-3&cid=555&t=event&ec=fast-ai&ea=fast-ai-salary-range-prediction&dt=fast-ai-salary-range-prediction)

In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
!pip install -q -U 'fastai<=1.0.61'

You should consider upgrading via the '/usr/local/anaconda3/envs/dev-py3/bin/python -m pip install --upgrade pip' command.


In [3]:
from fastai.tabular import *

## Prepare Training Data

In [4]:
path = untar_data(URLs.ADULT_SAMPLE)
df = pd.read_csv(path/'adult.csv')

In [5]:
dep_var = 'salary'
cat_names = ['workclass', 'education', 'marital-status', 'occupation', 'relationship', 'race']
cont_names = ['age', 'fnlwgt', 'education-num']
procs = [FillMissing, Categorify, Normalize]

In [6]:
test = TabularList.from_df(df.iloc[800:1000].copy(), path=path, cat_names=cat_names, cont_names=cont_names)

In [7]:
data = (TabularList.from_df(df, path=path, cat_names=cat_names, cont_names=cont_names, procs=procs)
                           .split_by_idx(list(range(800,1000)))
                           .label_from_df(cols=dep_var)
                           .add_test(test)
                           .databunch())

In [8]:
data.show_batch(rows=10)

## Model Training

In [9]:
learn = tabular_learner(data, layers=[200,100], metrics=accuracy)

In [10]:
learn.fit(1, 1e-2)

In [11]:
row = df.iloc[0] # sample input date for testing

learn.predict(row)

(Category tensor(1), tensor(1), tensor([0.4633, 0.5367]))

## Create BentoService for model serving

In [12]:
%%writefile tabular_csv.py

from bentoml import env, api, artifacts, BentoService
from bentoml.frameworks.fastai import Fastai1ModelArtifact
from bentoml.adapters import DataframeInput


@env(pip_packages=['fastai'])
@artifacts([Fastai1ModelArtifact('model')])
class FastaiTabularModel(BentoService):
    
    @api(input=DataframeInput(), batch=True)
    def predict(self, df):
        results = []
        for _, row in df.iterrows():       
            prediction = self.artifacts.model.predict(row)
            results.append(prediction[0].obj)
        return results

Overwriting tabular_csv.py


## Save BentoService to file archive

In [13]:
# 1) import the custom BentoService defined above
from tabular_csv import FastaiTabularModel

# 2) `pack` it with required artifacts
svc = FastaiTabularModel()
svc.pack('model', learn)

# 3) save your BentoSerivce
saved_path = svc.save()

[2020-10-01 14:45:17,839] WARNING - Using BentoML installed in `editable` model, the local BentoML repository including all code changes will be packaged together with saved bundle created, under the './bundled_pip_dependencies' directory of the saved bundle.
[2020-10-01 14:45:18,011] INFO - Using default docker base image: `None` specified inBentoML config file or env var. User must make sure that the docker base image either has Python 3.7 or conda installed.
[2020-10-01 14:45:18,013] WARNING - BentoML by default does not include spacy and torchvision package when using FastaiModelArtifact. To make sure BentoML bundle those packages if they are required for your model, either import those packages in BentoService definition file or manually add them via `@env(pip_packages=['torchvision'])` when defining a BentoService
[2020-10-01 14:45:18,015] WARNING - Overwriting existing pip package requirement 'fastai==1.0.61' to 'fastai<2.0.0'
[2020-10-01 14:45:18,517] INFO - Detected non-PyPI-r

no previously-included directories found matching 'e2e_tests'
no previously-included directories found matching 'tests'
no previously-included directories found matching 'benchmark'


UPDATING BentoML-0.9.1+1.g0655cf16.dirty/bentoml/_version.py
set BentoML-0.9.1+1.g0655cf16.dirty/bentoml/_version.py to '0.9.1+1.g0655cf16.dirty'
[2020-10-01 14:45:22,277] INFO - BentoService bundle 'FastaiTabularModel:20201001144518_0C60FD' saved to: /Users/bozhaoyu/bentoml/repository/FastaiTabularModel/20201001144518_0C60FD


## REST API Model Serving


To start a REST API model server with the BentoService saved above, use the bentoml serve command:

In [15]:
!bentoml serve FastaiTabularModel:latest

[2020-09-22 16:56:10,329] INFO - Getting latest version FastaiTabularModel:20200922163833_30289D
[2020-09-22 16:56:10,330] INFO - Starting BentoML API server in development mode..
[2020-09-22 16:56:10,622] WARNING - Using BentoML installed in `editable` model, the local BentoML repository including all code changes will be packaged together with saved bundle created, under the './bundled_pip_dependencies' directory of the saved bundle.
[2020-09-22 16:56:10,640] WARNING - Saved BentoService bundle version mismatch: loading BentoService bundle create with BentoML version 0.9.0.pre, but loading from BentoML version 0.9.0.pre+3.gcebf2015
[2020-09-22 16:56:11,014] INFO - Using default docker base image: `None` specified inBentoML config file or env var. User must make sure that the docker base image either has Python 3.7 or conda installed.
[2020-09-22 16:56:13,998] WARNING - BentoML by default does not include spacy and torchvision package when using FastaiModelArtifact. To make sure Bento

If you are running this notebook from Google Colab, you can start the dev server with `--run-with-ngrok` option, to gain acccess to the API endpoint via a public endpoint managed by [ngrok](https://ngrok.com/):

In [ ]:
!bentoml serve FastaiTabularModel:latest --run-with-ngrok

### Send prediction requeset to the REST API server

#### JSON Request

```bash
curl -X POST \
  http://localhost:5000/predict \
  -H 'Content-Type: application/json' \
  -d '[{
  "age": 49,
  "workclass": "Private",
  "fnlwgt": 101320,
  "education": "Assoc-acdm",
  "education-num": 12.0,
  "marital-status": "Married-civ-spouse",
  "occupation": "",
  "relationship": "Wift",
  "race": "White",
  "sex": "Female",
  "capital-gain": 0,
  "capital-loss": 1902,
  "hours-per-week": 40,
  "native-country": "United-States",
  "salary": ">=50k"
}]'
```

#### CSV Request

```bash
curl -X POST "http://127.0.0.1:5000/predict" \
    -H "Content-Type: text/csv" \
    --data-binary @test.csv
```

## Containerize model server with Docker


One common way of distributing this model API server for production deployment, is via Docker containers. And BentoML provides a convenient way to do that.

Note that docker is **not available in Google Colab**. You will need to download and run this notebook locally to try out this containerization with docker feature.

If you already have docker configured, simply run the follow command to product a docker container serving the IrisClassifier prediction service created above:

In [16]:
!bentoml containerize FastaiTabularModel:latest

[2020-09-22 16:56:43,434] INFO - Getting latest version FastaiTabularModel:20200922163833_30289D
Found Bento: /Users/bozhaoyu/bentoml/repository/FastaiTabularModel/20200922163833_30289D
[2020-09-22 16:56:43,475] WARNING - Using BentoML installed in `editable` model, the local BentoML repository including all code changes will be packaged together with saved bundle created, under the './bundled_pip_dependencies' directory of the saved bundle.
[2020-09-22 16:56:43,496] WARNING - Saved BentoService bundle version mismatch: loading BentoService bundle create with BentoML version 0.9.0.pre, but loading from BentoML version 0.9.0.pre+3.gcebf2015
Tag not specified, using tag parsed from BentoService: 'fastaitabularmodel:20200922163833_30289D'
Building Docker image fastaitabularmodel:20200922163833_30289D from FastaiTabularModel:latest 
-we in here
processed docker file
(None, None)
root in create archive /Users/bozhaoyu/bentoml/repository/FastaiTabularModel/20200922163833_30289D ['Dockerfile'

In [17]:
!docker run -p 5000:5000 fastaitabularmodel:20200922163833_30289D

[2020-09-23 00:01:08,992] INFO - Starting BentoML API server in production mode..
[2020-09-23 00:01:09,478] INFO - get_gunicorn_num_of_workers: 3, calculated by cpu count
[2020-09-23 00:01:09 +0000] [1] [INFO] Starting gunicorn 20.0.4
[2020-09-23 00:01:09 +0000] [1] [INFO] Listening at: http://0.0.0.0:5000 (1)
[2020-09-23 00:01:09 +0000] [1] [INFO] Using worker: sync
[2020-09-23 00:01:09 +0000] [12] [INFO] Booting worker with pid: 12
[2020-09-23 00:01:09 +0000] [13] [INFO] Booting worker with pid: 13
[2020-09-23 00:01:09 +0000] [14] [INFO] Booting worker with pid: 14
[2020-09-23 00:01:09,734] WARNING - Using BentoML not from official PyPI release. In order to find the same version of BentoML when deploying your BentoService, you must set the 'core/bentoml_deploy_version' config to a http/git location of your BentoML fork, e.g.: 'bentoml_deploy_version = git+https://github.com/{username}/bentoml.git@{branch}'
[2020-09-23 00:01:09,736] WARNING - Using BentoML not from official PyPI relea

## Load saved BentoService

bentoml.load is the API for loading a BentoML packaged model in python:

In [14]:
from bentoml import load

svc = load(saved_path)
print(svc.predict(df.iloc[0:1]))

[2020-10-01 14:45:29,278] WARNING - Saved BentoService bundle version mismatch: loading BentoService bundle create with BentoML version 0.9.1, but loading from BentoML version 0.9.1+1.g0655cf16.dirty
[2020-10-01 14:45:29,279] WARNING - Module `tabular_csv` already loaded, using existing imported module.
[2020-10-01 14:45:29,286] WARNING - pip package requirement pandas already exist
[2020-10-01 14:45:29,287] WARNING - BentoML by default does not include spacy and torchvision package when using FastaiModelArtifact. To make sure BentoML bundle those packages if they are required for your model, either import those packages in BentoService definition file or manually add them via `@env(pip_packages=['torchvision'])` when defining a BentoService
[2020-10-01 14:45:29,288] WARNING - pip package requirement torch already exist
[2020-10-01 14:45:29,290] WARNING - pip package requirement fastai<2.0.0 already exist
['>=50k']


## Launch inference job from CLI

BentoML cli supports loading and running a packaged model from CLI. With the DataframeInput adapter, the CLI command supports reading input Dataframe data from CLI argument or local csv or json files:

In [ ]:
!bentoml run FastaiTabularModel:latest predict \
    --input-file https://raw.githubusercontent.com/bentoml/gallery/master/fast-ai/salary-range-prediction/test.csv

# Deployment Options

If you are at a small team with limited engineering or DevOps resources, try out automated deployment with BentoML CLI, currently supporting AWS Lambda, AWS SageMaker, and Azure Functions:
- [AWS Lambda Deployment Guide](https://docs.bentoml.org/en/latest/deployment/aws_lambda.html)
- [AWS SageMaker Deployment Guide](https://docs.bentoml.org/en/latest/deployment/aws_sagemaker.html)
- [Azure Functions Deployment Guide](https://docs.bentoml.org/en/latest/deployment/azure_functions.html)

If the cloud platform you are working with is not on the list above, try out these step-by-step guide on manually deploying BentoML packaged model to cloud platforms:
- [AWS ECS Deployment](https://docs.bentoml.org/en/latest/deployment/aws_ecs.html)
- [Google Cloud Run Deployment](https://docs.bentoml.org/en/latest/deployment/google_cloud_run.html)
- [Azure container instance Deployment](https://docs.bentoml.org/en/latest/deployment/azure_container_instance.html)
- [Heroku Deployment](https://docs.bentoml.org/en/latest/deployment/heroku.html)

Lastly, if you have a DevOps or ML Engineering team who's operating a Kubernetes or OpenShift cluster, use the following guides as references for implementating your deployment strategy:
- [Kubernetes Deployment](https://docs.bentoml.org/en/latest/deployment/kubernetes.html)
- [Knative Deployment](https://docs.bentoml.org/en/latest/deployment/knative.html)
- [Kubeflow Deployment](https://docs.bentoml.org/en/latest/deployment/kubeflow.html)
- [KFServing Deployment](https://docs.bentoml.org/en/latest/deployment/kfserving.html)
- [Clipper.ai Deployment Guide](https://docs.bentoml.org/en/latest/deployment/clipper.html)

